# package

In [55]:
import json, os, random, time, math, multiprocessing, traceback
import boto3
import sagemaker
from sagemaker import image_uris, get_execution_role
from sagemaker.session import production_variant
from pyarrow import parquet
from datetime import datetime, timedelta
import pickle

In [56]:
HOME = '/home/sagemaker-user/'

In [57]:
s3_cli = boto3.client('s3')
sm_sess = sagemaker.Session()
print('aws region:', sm_sess.boto_region_name)
sm_cli = boto3.client('sagemaker')
role = get_execution_role()
print('role:', role)

aws region: ap-southeast-1
role: arn:aws:iam::580243821396:role/service-role/AmazonSageMaker-ExecutionRole-20241030T174232


In [61]:
s3_model_s3 = 's3://warehouse-algo/rec/all_ctr_din_prod_1day_1121_test/job/Job-laidehe-test-all-ctr-din-prod-1day-1121-test-11-21-11-24-43/model'
s3_model_local = HOME + '/model/model_local_pkg/'
s3_model_local_tar_file = s3_model_local + 'model.tar.gz'
s3_model_online = 's3://warehouse-algo/rec/model_test/'
s3_model_online_tar_file = s3_model_online + 'model.tar.gz'
code_file_name = s3_model_local + '/code/inference.py'

'/home/sagemaker-user//model/model_local_pkg/model.tar.gz'

In [36]:
# ! mkdir {s3_model_local} 
! aws s3 cp --recursive {s3_model_s3} {s3_model_local}
# ! mkdir code

download: s3://warehouse-algo/rec/all_ctr_din_prod_1day_1121_test/job/Job-laidehe-test-all-ctr-din-prod-1day-1121-test-11-21-11-24-43/model/checkpoint to model/model_local_pkg/checkpoint
download: s3://warehouse-algo/rec/all_ctr_din_prod_1day_1121_test/job/Job-laidehe-test-all-ctr-din-prod-1day-1121-test-11-21-11-24-43/model/1732159550/variables/variables.index to model/model_local_pkg/1732159550/variables/variables.index
download: s3://warehouse-algo/rec/all_ctr_din_prod_1day_1121_test/job/Job-laidehe-test-all-ctr-din-prod-1day-1121-test-11-21-11-24-43/model/model.ckpt-5.index to model/model_local_pkg/model.ckpt-5.index
download: s3://warehouse-algo/rec/all_ctr_din_prod_1day_1121_test/job/Job-laidehe-test-all-ctr-din-prod-1day-1121-test-11-21-11-24-43/model/1732159550/variables/variables.data-00000-of-00001 to model/model_local_pkg/1732159550/variables/variables.data-00000-of-00001
download: s3://warehouse-algo/rec/all_ctr_din_prod_1day_1121_test/job/Job-laidehe-test-all-ctr-din-prod-

In [43]:
%%writefile {code_file_name}

# coding: utf-8

import json
import logging
import pickle
import os

logging.info('[DEBUG] current dir: %s %s', os.getcwd(), os.listdir("/opt/ml/model/"))

inputs3 = {
	"cate_level1_id": ["1"],
	"cate_level2_id": ["1"],
	"cate_level3_id": ["1"],
	"cate_level4_id": ["1"],
	"country": ["IN"],
	"ctr_7d": [0.1],
	"cvr_7d": [0.1],
	"show_7d": [100],
	"click_7d": [100],
	"cart_7d": [100],
	"ord_total": [100],
	"pay_total": [100],
	"ord_7d": [100],
	"pay_7d": [100]

}
# inputs3 = {"a":"b"}

# ipt4 = {"signature_name": "serving_default","instances": [inputs3,inputs3] }
# ipt4 = {"signature_name": "serving_default","inputs": [inputs3,inputs3] }
ipt4 = {"signature_name": "predict","instances": [inputs3,inputs3] }

def input_handler(data, context):
    """Pre-process request input before it is sent to TensorFlow Serving REST API
    Args:
        data (obj): the request data stream
        context (Context): an object containing request and configuration details
    Returns:
        (dict): a JSON-serializable dict that contains request body and headers
    """

    if context.request_content_type == "application/json":
        # logging.info('[DEBUG] request_data1: %s %s', context.request_content_type, '')
        # d = data.read().decode('utf-8')
        logging.info('[DEBUG] request_data2: %s %s', context.request_content_type, data)
        # d = data.read().decode('utf-8')
        new_data = json.dumps(ipt4).encode('utf-8')
        return new_data

    # raise ValueError('Invalid req type: %s' % context.request_content_type)


def output_handler(data, context):
    """Post-process TensorFlow Serving output before it is returned to the client.
    Args:
        data (obj): the TensorFlow serving response
        context (Context): an object containing request and configuration details
    Returns:
        (bytes, string): data to return to client, response content type
    """
    logging.info('[DEBUG] output_data: %s %s  %s', type(data), data, context)
    # logging.info('[DEBUG] output_data_content: %s', data.content.decode('utf-8'))
    # if data.status_code != 200:
    #     raise ValueError(data.content.decode('utf-8'))

    response_content_type = context.accept_header
    prediction = data.content
    return prediction, response_content_type



Writing /home/sagemaker-user//model/model_local_pkg//code/inference.py


In [50]:
! tar -czvf {s3_model_local_tar_file}  {s3_model_local}

tar: Removing leading `/' from member names
/home/sagemaker-user//model/model_local_pkg/
/home/sagemaker-user//model/model_local_pkg/code/
/home/sagemaker-user//model/model_local_pkg/code/inference.py
/home/sagemaker-user//model/model_local_pkg/.ipynb_checkpoints/
/home/sagemaker-user//model/model_local_pkg/1732159550/
/home/sagemaker-user//model/model_local_pkg/1732159550/variables/
/home/sagemaker-user//model/model_local_pkg/1732159550/variables/variables.index
/home/sagemaker-user//model/model_local_pkg/1732159550/variables/variables.data-00000-of-00001
/home/sagemaker-user//model/model_local_pkg/1732159550/saved_model.pb
/home/sagemaker-user//model/model_local_pkg/eval/
/home/sagemaker-user//model/model_local_pkg/eval/events.out.tfevents.1732159541.ip-10-0-216-192.ap-southeast-1.compute.internal
/home/sagemaker-user//model/model_local_pkg/checkpoint
/home/sagemaker-user//model/model_local_pkg/model.ckpt-5.index
/home/sagemaker-user//model/model_local_pkg/model.ckpt-0.index
/home/sa

In [51]:
! aws s3 cp {s3_model_local_tar_file} {s3_model_online}

upload: model/model_local_pkg/model.tar.gz to s3://warehouse-algo/rec/model_test/model.tar.gz


# deploy

In [62]:
def deploy_new_endpoint(model_data,
                        endpoint_name,
                        instance_type='ml.r5.large',
                        instance_count=1,
                        retry_times=0):
    # If an endpoint could describe, it exists, and can not be created by deploy.
    try:
        print(client.describe_endpoint(EndpointName=endpoint_name))
        return
    except:
        pass

    # edp_model_name = endpoint_name + '-' + str(random.randint(10000, 19999))
    variant_name = "Variant-xlarge-1" # start from 1, incr 1 when updating.
    img = sagemaker.image_uris.retrieve(
        framework='tensorflow',
        version='1.15',
        region=sm_sess.boto_region_name,
        image_scope='inference',
        instance_type=instance_type
    )

    sm_sess.create_model(
        name=endpoint_name,
        role=role,
        container_defs={
            "Image": img,
            "ModelDataUrl": model_data,
            'Environment': {
                'TF_DISABLE_MKL': '1',
                'TF_DISABLE_POOL_ALLOCATOR': '1',
                # 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code/',  # Directory inside the container
                # 'SAGEMAKER_PROGRAM': 'inference.py', 
            },
        }
    )

    

    variant1 = production_variant(
        model_name=endpoint_name,
        instance_type=instance_type,
        initial_instance_count=instance_count,
        variant_name=variant_name,
        initial_weight=1,
    )

    sm_sess.endpoint_from_production_variants(
        name=endpoint_name, production_variants=[variant1],
        tags=[{'Key': 'cost-team', 'Value': 'algorithm'}],
    )
    print(sm_cli.describe_endpoint(EndpointName=endpoint_name))
    # wait_edp_inservice(endpoint_name)

endpoint =  'ctr-model-debug1121'
model_data = s3_model_online_tar_file
deploy_new_endpoint(model_data=model_data, endpoint_name=endpoint)

----!{'EndpointName': 'ctr-model-debug1121', 'EndpointArn': 'arn:aws:sagemaker:ap-southeast-1:580243821396:endpoint/ctr-model-debug1121', 'EndpointConfigName': 'ctr-model-debug1121', 'ProductionVariants': [{'VariantName': 'Variant-xlarge-1', 'DeployedImages': [{'SpecifiedImage': '763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/tensorflow-inference:1.15-cpu', 'ResolvedImage': '763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/tensorflow-inference@sha256:6da74b9a715efe9abffd8ef715f7b7ee2338bc8f8f1ba2cf4fedf47843bffa76', 'ResolutionTime': datetime.datetime(2024, 11, 21, 11, 10, 42, 718000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'CreationTime': datetime.datetime(2024, 11, 21, 11, 10, 42, 215000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2024, 11, 21, 11, 13, 11, 121000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '491a3a36-c18b-46a1-bb68-a3cb56d350dd', 

# request

In [63]:
import copy
import json
import time

import boto3

inputs = {
	"inputs": {
		"price": ["1", "2", "3", "4"],
		"rebuy": [1, 1, 1, 1],
		"country": "Savana_IN",
		"query": "ruffle placket blouse",
		"goods_id": [1295132, 119253, 1315492, 1233092],
		"uuid": "___X_911b557d-8ba5-4ec7-8637-cae016809ac0-1712127888282",
		"cid4": [522, 522, 522, 607],
		"cid2": [12, 12, 12, 13],
		"cid3": [80, 80, 80, 96],
		"cid1": [2, 2, 2, 809]
	}
}

inputs2 = {
	"inputs": {
        "inputs": {
            "goods_id": ["1"]
        }
    }
		
}

inputs3 = {
	"cate_level1_id": ["1"],
	"cate_level2_id": ["1"],
	"cate_level3_id": ["1"],
	"cate_level4_id": ["1"],
	"country": ["IN"],
	"ctr_7d": [0.1],
	"cvr_7d": [0.1],
	"show_7d": [100],
	"click_7d": [100],
	"cart_7d": [100],
	"ord_total": [100],
	"pay_total": [100],
	"ord_7d": [100],
	"pay_7d": [100],
	"seq_goods_id": ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"],
	"goods_id": ["1"],
	"seq_cate_id": ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"],
	"cate_id": ["1"]
}

inputs3 = {
	"cate_level1_id": ["1"],
	"cate_level2_id": ["1"],
	"cate_level3_id": ["1"],
	"cate_level4_id": ["1"],
	"country": ["IN"],
	"ctr_7d": [0.1],
	"cvr_7d": [0.1],
	"show_7d": [100],
	"click_7d": [100],
	"cart_7d": [100],
	"ord_total": [100],
	"pay_total": [100],
	"ord_7d": [100],
	"pay_7d": [100]

}
# inputs3 = {"a":"b"}

# ipt4 = {"signature_name": "serving_default","instances": [inputs3,inputs3] }
# ipt4 = {"signature_name": "serving_default","inputs": [inputs3,inputs3] }
ipt4 = {"signature_name": "predict","instances": [inputs3,inputs3] }
# ipt4 = {"inputs": [inputs3,inputs3] }
# ipt4 = {"signature_name": "serving_default","inputs": [{"a":"b"},{"a":"b"}] }

# edp_name_exp = "all-ctr-din-prod-1day-edp"
# edp_name_exp = "all-ctr-din-prod-1day-edp-testv3"
# edp_name_exp = "all-ctr-din-prod-1day-edp-testv5"
# edp_name_exp = "all-ctr-din-prod-1day-edp-testv7"

sg_client = boto3.client("sagemaker-runtime")

In [67]:
print('inp-json-dump', json.dumps(ipt4))

res = sg_client.invoke_endpoint(
        EndpointName=endpoint,
        Body=json.dumps(ipt4),
        # Body=json.dumps(ipt4).encode('utf-8'),
    # .encode('utf-8'),
        # Body=ipt4,
        ContentType="application/json"
    )
print(res["Body"].read())
# res_json = json.loads(res["Body"].read())
# print(res_json)
# res_json['predictions'][0]['probabilities'][0]

inp-json-dump {"signature_name": "predict", "instances": [{"cate_level1_id": ["1"], "cate_level2_id": ["1"], "cate_level3_id": ["1"], "cate_level4_id": ["1"], "country": ["IN"], "ctr_7d": [0.1], "cvr_7d": [0.1], "show_7d": [100], "click_7d": [100], "cart_7d": [100], "ord_total": [100], "pay_total": [100], "ord_7d": [100], "pay_7d": [100]}, {"cate_level1_id": ["1"], "cate_level2_id": ["1"], "cate_level3_id": ["1"], "cate_level4_id": ["1"], "country": ["IN"], "ctr_7d": [0.1], "cvr_7d": [0.1], "show_7d": [100], "click_7d": [100], "cart_7d": [100], "ord_total": [100], "pay_total": [100], "ord_7d": [100], "pay_7d": [100]}]}
b'{\n    "predictions": [\n        {\n            "probabilities": [0.465676159, 0.534323812],\n            "logits": [0.137511566],\n            "class_ids": [1],\n            "classes": ["1"],\n            "all_class_ids": [0, 1],\n            "logistic": [0.534323812],\n            "all_classes": ["0", "1"]\n        },\n        {\n            "probabilities": [0.46567